In [1]:
import numpy as np
import pandas as pd
import glob 

In [2]:
cur_path='/home/alandoli/Desktop/CP_Lim'
# link = https://www.kaggle.com/code/rkuo2000/induction-motor-faults-detection/data

In [3]:
normal_file_names = glob.glob(cur_path+'/normal/*.csv')
imnormal_file_names_6g = glob.glob(cur_path+'/imbalance/6g/*.csv')
imnormal_file_names_10g = glob.glob(cur_path+'/imbalance/10g/*.csv')
imnormal_file_names_15g = glob.glob(cur_path+'/imbalance//15g/*.csv')
imnormal_file_names_20g = glob.glob(cur_path+'/imbalance//20g/*.csv')
imnormal_file_names_25g = glob.glob(cur_path+'/imbalance//25g/*.csv')
imnormal_file_names_30g = glob.glob(cur_path+'/imbalance//30g/*.csv')

In [4]:
def dataReader(path_names):
    data_n = pd.DataFrame()
    for i in path_names:
        low_data = pd.read_csv(i,header=None)
        data_n = pd.concat([data_n,low_data],ignore_index=True)
    return data_n

In [5]:
data_n = dataReader(normal_file_names)
data_6g = dataReader(imnormal_file_names_6g)
data_10g = dataReader(imnormal_file_names_10g)
data_15g = dataReader(imnormal_file_names_15g)
data_20g = dataReader(imnormal_file_names_20g)
data_25g = dataReader(imnormal_file_names_25g)
data_30g = dataReader(imnormal_file_names_30g)

In [6]:
def downSampler(data,a,b):
    """
    data = data
    a = start index
    b = sampling rate
    """
    data_decreased = pd.DataFrame()
    x = b
    for i in range(int(len(data)/x)):
        data_decreased = data_decreased.append(data.iloc[a:b,:].sum()/x,ignore_index=True)
        a += x
        b += x
    return data_decreased

In [7]:
Sampler=70
data_n1 = downSampler(data_n, 0, Sampler)
data_6g1 = downSampler(data_6g, 0, Sampler)
data_10g1 = downSampler(data_10g, 0, Sampler)
data_15g1 = downSampler(data_15g, 0, Sampler)
data_20g1 = downSampler(data_20g, 0, Sampler)
data_25g1 = downSampler(data_25g, 0, Sampler)
data_30g1 = downSampler(data_30g, 0, Sampler)



In [8]:
from scipy import signal
def FFT(data):
    autocorr = signal.fftconvolve(data,data[::-1],mode='full')
    return pd.DataFrame(autocorr)

In [9]:
data_n1 = FFT(data_n1)
data_6g1 = FFT(data_6g1)
data_10g1 = FFT(data_10g1)
data_15g1 = FFT(data_15g1)
data_20g1 = FFT(data_20g1)
data_25g1 = FFT(data_25g1)
data_30g1 = FFT(data_30g1)

In [10]:
y_1 = pd.DataFrame(np.ones(int(len(data_n1)),dtype=int))
y_2 = pd.DataFrame(np.zeros(int(len(data_6g1)),dtype=int))
y_3 = pd.DataFrame(np.full((int(len(data_10g1)),1),2))
y_4 = pd.DataFrame(np.full((int(len(data_15g1)),1),3))
y_5 = pd.DataFrame(np.full((int(len(data_20g1)),1),4))
y_6 = pd.DataFrame(np.full((int(len(data_25g1)),1),5))
y_7 = pd.DataFrame(np.full((int(len(data_30g1)),1),6))
y = pd.concat([y_1,y_2,y_3,y_4,y_5,y_6,y_7], ignore_index=True)
y

,0
0,1
1,1
2,1
3,1
4,1
...,...
2407128,6
2407129,6
2407130,6
2407131,6


In [11]:
data = pd.concat([data_n1,data_6g1,data_10g1,data_15g1,data_20g1,data_25g1,data_30g1],ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2407133 entries, 0 to 2407132
Data columns (total 15 columns):
 #   Column  Dtype  
---  ------  -----  
 0   0       float64
 1   1       float64
 2   2       float64
 3   3       float64
 4   4       float64
 5   5       float64
 6   6       float64
 7   7       float64
 8   8       float64
 9   9       float64
 10  10      float64
 11  11      float64
 12  12      float64
 13  13      float64
 14  14      float64
dtypes: float64(15)
memory usage: 275.5 MB


In [12]:
path="fault_data/L10/"
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.20, shuffle=True)
print("Shape of Train Data : {}".format(X_train.shape))
print("Shape of Test Data : {}".format(X_test.shape))
np.save(path+"X_train", X_train)
np.save(path+"X_test", X_test)
np.save(path+"y_train", y_train)
np.save(path+"y_test", y_test)

Shape of Train Data : (1925706, 15)
Shape of Test Data : (481427, 15)
